In [1]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [14]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='./data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           )

test_dataset = torchvision.datasets.MNIST(root='./data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

In [28]:
test_dataset?

In [7]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)


In [20]:
test_loader?

In [35]:
for img, label in (test_loader):
    a,b = img, label
    break

In [47]:
img.size()

torch.Size([100, 1, 28, 28])

In [10]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out


In [11]:
model = ConvNet(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [12]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


/home2/kanakala.ganesh/miniconda3/envs/geometric/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448238472/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch [1/5], Step [100/600], Loss: 0.1152
Epoch [1/5], Step [200/600], Loss: 0.1516
Epoch [1/5], Step [300/600], Loss: 0.0628
Epoch [1/5], Step [400/600], Loss: 0.0147
Epoch [1/5], Step [500/600], Loss: 0.0866
Epoch [1/5], Step [600/600], Loss: 0.0265
Epoch [2/5], Step [100/600], Loss: 0.0087
Epoch [2/5], Step [200/600], Loss: 0.0286
Epoch [2/5], Step [300/600], Loss: 0.0297
Epoch [2/5], Step [400/600], Loss: 0.0210
Epoch [2/5], Step [500/600], Loss: 0.0136
Epoch [2/5], Step [600/600], Loss: 0.0280
Epoch [3/5], Step [100/600], Loss: 0.0141
Epoch [3/5], Step [200/600], Loss: 0.1590
Epoch [3/5], Step [300/600], Loss: 0.0785
Epoch [3/5], Step [400/600], Loss: 0.0117
Epoch [3/5], Step [500/600], Loss: 0.0278
Epoch [3/5], Step [600/600], Loss: 0.0041
Epoch [4/5], Step [100/600], Loss: 0.0073
Epoch [4/5], Step [200/600], Loss: 0.0041
Epoch [4/5], Step [300/600], Loss: 0.0215
Epoch [4/5], Step [400/600], Loss: 0.0343
Epoch [4/5], Step [500/600], Loss: 0.0087
Epoch [4/5], Step [600/600], Loss:

In [49]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        print(outputs)
        break
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

#     print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))


tensor([[-1.3519e+01, -5.3544e+00, -2.4464e+00,  3.2097e+00, -9.2771e+00,
         -7.1392e+00, -2.3539e+01,  1.7636e+01, -9.7450e+00, -1.2005e+00],
        [-4.0440e+00, -3.1912e+00,  1.2936e+01, -5.7672e+00, -1.1353e+01,
         -1.1220e+01, -5.6841e+00, -1.1401e+01, -9.5698e+00, -1.2612e+01],
        [-7.2942e+00,  1.0029e+01, -4.6625e+00, -6.6914e+00,  4.6175e-01,
         -3.1672e+00, -4.4250e+00,  6.1398e-01, -4.2955e+00, -5.3485e+00],
        [ 9.8972e+00, -1.3772e+01, -2.3096e+00, -7.3914e+00, -8.5755e+00,
         -6.9668e+00, -1.8110e+00, -6.2960e+00, -5.3358e+00, -4.5864e+00],
        [-9.2748e+00, -4.8855e+00, -6.9033e+00, -1.1640e+01,  1.0280e+01,
         -7.6036e+00, -7.6754e+00, -3.3487e+00, -8.7016e+00,  1.1962e+00],
        [-7.9378e+00,  1.0670e+01, -4.3413e+00, -8.3429e+00,  8.1663e-01,
         -5.8374e+00, -5.5949e+00,  2.0568e+00, -3.4792e+00, -4.9962e+00],
        [-2.1466e+01, -7.2714e+00, -8.1817e+00, -9.4251e+00,  7.0200e+00,
         -5.2214e+00, -1.5783e+0

In [51]:
outputs.size()

torch.Size([100, 10])

In [54]:
images.size()

torch.Size([100, 1, 28, 28])

In [55]:
labels

tensor([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5,
        4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 2,
        4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4, 6, 4, 3, 0, 7, 0,
        2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3, 6, 1, 3, 6, 9, 3, 1, 4,
        1, 7, 6, 9], device='cuda:0')

In [52]:
_, predicted = torch.max(outputs.data, 1)

In [53]:
predicted

tensor([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5,
        4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 2,
        4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4, 6, 4, 3, 0, 7, 0,
        2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3, 6, 1, 3, 6, 9, 3, 1, 4,
        1, 7, 6, 9], device='cuda:0')

In [56]:
(predicted == labels).sum().item()

100